In [5]:
import numpy as np
import pandas as pd
from scipy import interpolate as intp
from scipy import optimize
import matplotlib.pyplot as plt
import gsw

In [8]:
glodap = pd.read_csv('GLODAPv2.2019_Merged_Master_File.csv')

In [9]:
cruises = glodap.cruise.unique()
# This will be looped over cruises
cruise_df  = glodap[glodap.cruise == cruises[0]]
# This will be a loop over stations
stations = cruise_df.station.unique()
station_df = cruise_df[ cruise_df.station == stations[0] ]

In [10]:
cruise_df

,cruise,station,cast,year,month,day,hour,minute,latitude,longitude,...,toc,tocf,doc,docf,don,donf,tdn,tdnf,chla,chlaf
0,1.0,319.0,1.0,1984.0,7.0,20.0,14.0,46.0,80.567,7.2267,...,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0
1,1.0,319.0,1.0,1984.0,7.0,20.0,14.0,46.0,80.567,7.2267,...,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0
2,1.0,319.0,1.0,1984.0,7.0,20.0,14.0,46.0,80.567,7.2267,...,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0
3,1.0,319.0,1.0,1984.0,7.0,20.0,14.0,46.0,80.567,7.2267,...,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0
4,1.0,319.0,1.0,1984.0,7.0,20.0,14.0,46.0,80.567,7.2267,...,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,1.0,387.0,1.0,1984.0,8.0,5.0,19.0,38.0,76.333,12.9550,...,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0
758,1.0,387.0,1.0,1984.0,8.0,5.0,19.0,38.0,76.333,12.9550,...,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0
759,1.0,387.0,1.0,1984.0,8.0,5.0,19.0,38.0,76.333,12.9550,...,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0
760,1.0,387.0,1.0,1984.0,8.0,5.0,19.0,38.0,76.333,12.9550,...,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0,-9999.0,9.0


In [11]:
SA = gsw.SA_from_SP(station_df.salinity, station_df.pressure, station_df.longitude, station_df.latitude)
CT = gsw.CT_from_t(station_df.salinity,station_df.temperature,station_df.pressure)
sigma0 = gsw.sigma0(SA,CT)

In [20]:
def find_sigma0_z(salinity, temperature, pressure, latitude, longitude, sigmas):
    """
    Find the depth of the isopycnal using T/S from a cast
    Inputs:
        salinity    (array) : Salinity in PSU
        temperature (array) : In-situ temperature (C)
        latitude    (array) : Latitude of the sample
        longitude   (array) : Longitude of the sample
        sigmas      (array) : Isopycnals for which to find the depth
    Outputs:
        sigma_z     (array) : Depth of the isopycnals
    Algorithm:
        1. Convert in-situ salinity and temperature to Absolute Salinity and Conservative Temperature
        2. Calculate sigma0
        3. Check to ensure the isopycnal surface spans the density range of the water column
        4. Build interpolating functions for both T and S
    """    
    
    def dens_diff(pressure, SA_f, CT_f, target_sigma0):
        print(pressure)
        return np.square(gsw.sigma0(SA_f(pressure),CT_f(pressure)) - target_sigma0)
    
    # Promote to single element array if only one level requested
    
    is_scalar = type(sigmas) == type(0.)
    if is_scalar:
        sigmas = np.array([sigmas])
    
    pressure = np.array(pressure)
    SA = gsw.SA_from_SP(salinity, pressure, longitude, latitude)
    CT = gsw.CT_from_t(salinity, temperature, pressure)
    
    P_sort = np.unique(pressure)
    SA_sort = np.zeros(P_sort.shape)
    CT_sort = np.zeros(P_sort.shape)
    # Loop through all pressures that overlap, average the temperatures and salinity accordingly
    for idx, P in enumerate(P_sort):
        presidx = (pressure == P)        
        SA_sort[idx] = SA[presidx].mean()
        CT_sort[idx] = CT[presidx].mean()        
    
    SA_intp = intp.interp1d(P_sort, SA_sort, kind='quadratic')
    CT_intp = intp.interp1d(P_sort, CT_sort, kind='quadratic')
        
    sigma0 = gsw.sigma0(SA_sort,CT_sort)
    sigma0_max = sigma0.max()
    sigma0_min = sigma0.min()
    
    sigma0_z = np.zeros(sigmas.shape)
    
    for sigidx, siglev in enumerate(sigmas):
        if (siglev < sigma0_min) or (siglev > sigma0_max):
            sigma0_z[sigidx] = np.nan
        else:
            for botidx in range(len(sigma0)-1):
                if (siglev >= sigma0[botidx]) & (siglev <= sigma0[botidx+1]):
                    start_idx = botidx
                    break
                                              
            sigma0_z[sigidx] = optimize.minimize_scalar(dens_diff,
                                                        bounds=[pressure[botidx],pressure[botidx+1]],
                                                        method='Bounded',
                                                        args=(SA_intp, CT_intp, siglev))

    if is_scalar:
        sigma0_z = np.squeeze(sigma0_z)
    
    return sigma0_z
    
    
    

In [21]:
find_sigma0_z(station_df.salinity,station_df.temperature,station_df.pressure,station_df.latitude,station_df.longitude,27.)

8.6


TypeError: float() argument must be a string or a number, not 'OptimizeResult'